# CollateralisedCrossCurrencySwap

In this notebook, we illustrate the collateral and exposure simulation with DiffFusion.jl.

## Incorporating DiffFusion.jl

We setup a Julia project environment in the current directory.

In [ ]:
using Pkg
Pkg.activate("../.")

We need to incorporate the DiffFusion package.

In [ ]:
using DiffFusion

And use some further packages.

In [ ]:
using Plots
gr()
# plotlyjs()

## Model Setup

For this example, we a hybrid model. Hybrid model is loaded and build from the DiffFusion.jl *Examples* sub-module. 

The pre-specified model can easily be replaced by alternative models and model parameter settings.

In [ ]:
ex = DiffFusion.Examples.load("g3_1factor_flat")
ex = DiffFusion.Examples.build(ex)
ts = DiffFusion.Examples.term_structures(ex)

## Monte Carlo Simulation and Path Setup

We simulate state variables on a specified time grid and create a `Path` object for subsequent scenario pricing.

In [ ]:
times = 0.0:1.0/48:5.0
n_paths = 2^13
sim = DiffFusion.simple_simulation(ex["md/G3"], ex["ch/STD"], times, n_paths, with_progress_bar = false)
path = DiffFusion.path(sim, ts, ex["ct/STD"], DiffFusion.LinearPathInterpolation)

## Product Setup

Our cross currency swap is composed of individual coupons. The coupons are then aggregated into legs.

In [ ]:
coupon_times = 0.0:0.25:5.0

usd_coupons = [
    DiffFusion.CompoundedRateCoupon([s, e], [e-s], e, "USD:SOFR", nothing, nothing)
    for (s, e) in zip(coupon_times[1:end-1], coupon_times[2:end])
]
usd_floorlets = [
    DiffFusion.OptionletCoupon(cp, 0.0, -1.0) for cp in usd_coupons
]
usd_coupons_floored = [
    cp + fl for (cp, fl) in zip(usd_coupons, usd_floorlets)
]
eur_coupons = [
    DiffFusion.CompoundedRateCoupon([s, e], [e-s], e, "EUR:ESTR", nothing, 0.0050)
    for (s, e) in zip(coupon_times[1:end-1], coupon_times[2:end])
]

eur_leg = DiffFusion.cashflow_leg("leg/ESTR", eur_coupons, 10_000.00, "EUR:XCCY", "EUR-USD", +1.0)
#
usd_const_leg = DiffFusion.cashflow_leg("leg/SOFR/CN", usd_coupons, 10_000.00 * 1.07, "USD:SOFR", nothing, -1.0)
#
usd_mtm_leg = DiffFusion.mtm_cashflow_leg("leg/SOFR/MTM", usd_const_leg, 10_000.00, 0.0, "EUR:XCCY", "EUR-USD")

# final notional exchange are only used for ad-hoc comparison
eur_notional_leg = DiffFusion.cashflow_leg(
    "leg/EUR/NTL", 
    [ DiffFusion.FixedCashFlow(5.0, 1.0) ],
    10_000.00,
    "EUR:XCCY",
    "EUR-USD",
    -1.0,
)
usd_notional_leg = DiffFusion.cashflow_leg(
    "leg/USD/NTL", 
    [ DiffFusion.FixedCashFlow(5.0, 1.0) ],
    10_000.00 * 1.07,
    "USD:SOFR",
    nothing,
    +1.0,
)

A swap instrument is represented by a list of cash flow legs. The standard mark-to-market cross currency swap consists of the (constant notional) EUR leg and the USD leg with notional reset.

Other constellations and specifications can be activated by uncommenting the `swap` definition below.

In [ ]:
swap = [eur_leg, usd_mtm_leg]
#
# swap = [usd_const_floor_leg]
# swap = [eur_leg, usd_mtm_leg, usd_mtm_floor_leg]
# swap = [eur_leg, usd_const_leg, eur_notional_leg, usd_notional_leg]
# swap = [eur_leg, usd_const_leg]
# swap = [eur_notional_leg, usd_notional_leg]

## Scenario Calculation

We calculate un-discounted scenario prices for our reference swap.

In [ ]:
scens1 = DiffFusion.scenarios(swap, times, path, nothing, with_progress_bar = false)

## Collateral Simulation

A collateral balance account is simulated based on an initial `ScenarioCube`. The initial `ScenarioCube` represents the *netting set* over which  

In [ ]:
scens2 = DiffFusion.collateralised_portfolio(
    scens1,
    nothing, # fx_rates (ScenarioCube), if scenario prices need to be converted in foreign currency 
    times,
    0.0, # initial_collateral_balance
    0.0, # minimum_transfer_amount
    0.0, # threshold_amount
    0.0, # independent_amount
    2/48.0, # mpr
)

We can inspect the legs that are stored in the new scenario cube.

In [ ]:
scens2.leg_aliases

The first and second leg corresponds to the cross currency swap. The third leg represents the collateral balance over time.

We illustrate the collateral balance by plotting its mean over simulation time.

In [ ]:
scens2_agg = DiffFusion.aggregate(scens2, true, false)
plot(
    times,
    scens2_agg.X[1,:,3],
    title = "Collateral balance",
    label = scens2_agg.leg_aliases[3],
    xlabel = "simulation time",
    ylabel = "simulated value",
    size = (800, 600),
)

## Analysis of Collateralised Swap

We compare simulated paths and exposures for the collateralised swap and the uncollateralised swap.

In [ ]:
"""
Create an animated plot that compares analytics for two input scenarios
"""
function compare_scenarios(
    scens1::DiffFusion.ScenarioCube,
    scens2::DiffFusion.ScenarioCube;
    plot_title::String = "Scenario Analysis",
    sub_plot_title1::String = "first portfolio",
    sub_plot_title2::String = "second portfolio",
    xlabel::String = "observation time (years)",
    ylabel::String = "market value",
    xlims::Tuple = (0.0, 10.0),
    ylims::Tuple = (-10_000., +10_000.),
    font_size::Integer = 10,
    plot_size::Tuple = (600, 800),
    first_paths::Integer = 8,
    second_paths::Integer = 128,
    wait_seconds::Integer = 4,
    pfe_quantile = 0.95,
    line_width = 2,
    )
    #
    @info "Calculate analytics for first scenarios."
    scens1_agg = DiffFusion.aggregate(scens1, false, true)
    scens1_mv = DiffFusion.aggregate(scens1, true, true)
    scens1_ee = DiffFusion.expected_exposure(scens1, false, true, true)
    scens1_pfe = DiffFusion.potential_future_exposure(scens1, pfe_quantile)
    #
    @info "Calculate analytics for second scenarios."
    scens2_agg = DiffFusion.aggregate(scens2, false, true)
    scens2_mv = DiffFusion.aggregate(scens2, true, true)
    scens2_ee = DiffFusion.expected_exposure(scens2, false, true, true)
    scens2_pfe = DiffFusion.potential_future_exposure(scens2, 0.95)
    #
    a = Animation()
    #
    # we define some auxilliary functions...    
    #
    function wait(n_seconds, p)
        for i ∈ 1:n_seconds
            frame(a, p)
        end    
    end
    #
    function super_plot(p1, p2)
        return  plot(p1, p2,
            layout = (2,1),
            size = plot_size,
            plot_title = plot_title,
            plot_titlefontsize = font_size,
            left_margin = 5Plots.mm  # adjust this if xaxis label is cut off
        )
    end
    #
    function plot_paths(with_frame)
        p1 = plot(
            title = sub_plot_title1,
            titlefontsize = font_size,
            titlelocation = :right,
            guidefontsize = font_size,
            color_palette = :tab10
        )
        ylabel!(p1, ylabel)
        ylims!(p1, ylims...)
        #
        p2 = plot(
            title = sub_plot_title2,
            titlefontsize = font_size,
            titlelocation = :right,
            guidefontsize = font_size,
            color_palette = :tab10
        )
        xlabel!(p2, xlabel)
        ylabel!(p2, ylabel)
        ylims!(p2, ylims...)
        for i ∈ 1:first_paths
            plot!(p1, scens1_agg.times, scens1_agg.X[i,:,1], label="", lc=8)
            plot!(p2, scens2_agg.times, scens2_agg.X[i,:,1], label="", lc=8)
            p = super_plot(p1, p2)
            if with_frame
                frame(a, p)
            end
        end
        plot!(p1, scens1_agg.times, scens1_agg.X[first_paths+1:second_paths,:,1]', label="", lc=8)
        plot!(p2, scens2_agg.times, scens2_agg.X[first_paths+1:second_paths,:,1]', label="", lc=8)
        p = super_plot(p1, p2)
        if with_frame
            wait(wait_seconds, p)
        end
        return (p, p1, p2)
    end
    #
    @info "Build animation."
    # Full graph as first frame for PDF
    (p, p1, p2) = plot_paths(false)
    plot!(p1, scens1_pfe.times, scens1_pfe.X[1,:,1], label="potential future exposure",    lc=4, lw=line_width)
    plot!(p2, scens2_pfe.times, scens2_pfe.X[1,:,1], label="potential future exposure",    lc=4, lw=line_width)
    plot!(p1, scens1_ee.times,  scens1_ee.X[1,:,1],  label="expected (positive) exposure", lc=2, lw=line_width)
    plot!(p2, scens2_ee.times,  scens2_ee.X[1,:,1],  label="expected (positive) exposure", lc=2, lw=line_width)
    plot!(p1, scens1_mv.times,  scens1_mv.X[1,:,1],  label="expected market value",        lc=3, lw=line_width)
    plot!(p2, scens2_mv.times,  scens2_mv.X[1,:,1],  label="expected market value",        lc=3, lw=line_width)
    p = super_plot(p1, p2)
    wait(1, p)
    # intitial frames
    plot_paths(true)
    # expected market value
    (p, p1, p2) = plot_paths(false)
    plot!(p1, scens1_mv.times, scens1_mv.X[1,:,1], label="expected market value", lc=3, lw=line_width)
    plot!(p2, scens2_mv.times, scens2_mv.X[1,:,1], label="expected market value", lc=3, lw=line_width)
    p = super_plot(p1, p2)
    wait(4, p)
    # expected exposure
    (p, p1, p2) = plot_paths(false)
    plot!(p1, scens1_ee.times, scens1_ee.X[1,:,1], label="expected (positive) exposure", lc=2, lw=line_width)
    plot!(p2, scens2_ee.times, scens2_ee.X[1,:,1], label="expected (positive) exposure", lc=2, lw=line_width)
    plot!(p1, scens1_mv.times, scens1_mv.X[1,:,1], label="expected market value",        lc=3, lw=line_width)
    plot!(p2, scens2_mv.times, scens2_mv.X[1,:,1], label="expected market value",        lc=3, lw=line_width)
    p = super_plot(p1, p2)
    wait(4, p)
    # PFE
    (p, p1, p2) = plot_paths(false)
    plot!(p1, scens1_pfe.times, scens1_pfe.X[1,:,1], label="potential future exposure",    lc=4, lw=line_width)
    plot!(p2, scens2_pfe.times, scens2_pfe.X[1,:,1], label="potential future exposure",    lc=4, lw=line_width)
    plot!(p1, scens1_ee.times,  scens1_ee.X[1,:,1],  label="expected (positive) exposure", lc=2, lw=line_width)
    plot!(p2, scens2_ee.times,  scens2_ee.X[1,:,1],  label="expected (positive) exposure", lc=2, lw=line_width)
    plot!(p1, scens1_mv.times,  scens1_mv.X[1,:,1],  label="expected market value",        lc=3, lw=line_width)
    plot!(p2, scens2_mv.times,  scens2_mv.X[1,:,1],  label="expected market value",        lc=3, lw=line_width)
    p = super_plot(p1, p2)
    wait(10, p)
    @info "Plot GIF."
    gif(a, "CollateralisedCrossCurrencySwap.gif", fps = 1)
    # display(p)
end

In [ ]:
# calculate animation and plot graph
compare_scenarios(
    scens1,
    scens2,
    plot_title = "SOFR vs ESTR mark-to-market cross currency swap",
    sub_plot_title1 = "no collateral",
    sub_plot_title2 = "full collateral, two weeks MPoR",
    ylabel = "market value (USD, bp)",
    ylims = (-4000., 4000),
)


We find that expected market value for the uncollateralised and the collateralised swap is flat.

The mark-to-market feature of the modelled cross currency swap aims at mitigating counterparty credit risk (CCR). This property can be observed by the fact that expected exposure (EE) and potential future exposure (PFE) drop to zero at the quarterly reset dates.

However, prior to the reset dates the uncollateralised cross currency swap does exhibit relevant CCR expressed via EE and PFE.

From above analysis we see that collateralisation does reduce CCR. But with the modelled 2-weeks margin period of risk (MPoR), we still see PFE (and to lesser extend EE) spikes around the reset dates.